<a href="https://colab.research.google.com/github/vmokashi01/SSBU-Tiers/blob/master/SSBU_Tiers_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Super Smash Bros Ultimate Tier List Analysis

Super Smash Bros. Ultimate is a versatile fighting game that caters to both its casual and competitive communities. The game itself is composed of playable fighters, each with their distinct attacks and in-game stats, such as weight, gravity, and walking speed, which are then pitted against each other. The competitive community generates tier lists based on their sentiment regarding the ability for a fighter to win matches. Tier lists often implicitly include how well a fighter matches up against other characters, the skill cap of a fighter, as well as the strategies and attack combinations that can be used by the character. 

The aim of this exploration is to investigate the attributes of specific tiers within the most widely accepted SSBU tier list and to develop a model that can be used to predict the tier of a new character. 
 


1. Data Processing
2. Data Exploration
3. Modelling
4. Conclusions



In [0]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [61]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


## Data Processing

At this stage there are a few tasks that need to be completed in order to ensure that the data set can be worked with. I will be using the 'Character' column as a key between the tables - to do so, each table must be checked to ensure that every character is listed. As well, the names of each character must be written the same way. For example, 'Pokemon Trainer - Ivysaur' vs. 'PT [Ivysaur]' may cause issues when joining the tables. Next, the data types of each column should be changed to correctly reflect the data within the column. Lastly, empty columns should be dropped and the individual tables will be joined in order to view the relationships between every data point for each character.

Notes:
- Some characters are not listed in some tables. The data for these characters will either be filled through an additional data source or by taking the mean of characters within a similar weight class 
- Ice Climbers consist of two partenered characters (out of which one of them is player-controlled while the other one is a CPU); they have slightly different stats, however, they will be considered as a single unit by taking the mean of their individual data points and will always be viewed in their partnered form
- Some characters have alternate forms (such as Joker and Shulk), which shift their character data points; the exploration will consider their base data points


In [62]:
df = pd.read_csv('/content/drive/My Drive/SSBU-Tiers/tier_data.csv', delimiter=';')
df = df.drop(columns=['Unnamed: 4']) #drop empty column
df['Difficulty'] = df.apply(lambda x: x.Difficulty.replace('Difficulty: ', ''), axis=1) #clean up columns
df['Archetype'] = df.apply(lambda x: x.Archetype.replace(' Archetype', ''), axis=1)
df

,Character,Difficulty,Archetype,Tier
0,Banjo & Kazooie,Intermediate,Projectile,B
1,Bayonetta,Hard,Rushdown,D
2,Bowser,Easy,Heavy,B
3,Bowser Jr.,Easy,Rushdown,D
4,Byleth,Intermediate,Sword,B
...,...,...,...,...
77,Wolf,Easy,Projectile,A
78,Yoshi,Intermediate,Rushdown,B
79,Young Link,Intermediate,Projectile,A
80,Zelda,Easy,Projectile,B


In [0]:
def clean(table):
  table = table.iloc[:, :-1] #drop "Unnamed" columns
  if 'Rank' in table.columns:
    table = table.drop(columns=['Rank'])
  return table

In [0]:
tables = ['acceleration', 'air_speed', 'dash_speed', 'fall_speed', 'gravity', 'jump_duration', 'jump_height', 'landing', 'ledge', 'shield', 'walk_speed', 'weight']

In [67]:
for table in tables:
  table = pd.read_csv('/content/drive/My Drive/SSBU-Tiers/' + table + '.csv')
  table = clean(table)
  df.set_index('Character').join(table.set_index('Character'))

KeyError: ignored

In [65]:
weight = pd.read_csv('/content/drive/My Drive/SSBU-Tiers/weight.csv', delimiter=';')
weight = clean(weight)
weight
#df.set_index('Character').join(weight.set_index('Character'))

,Character,Weight
0,Bowser,135
1,King K. Rool,133
2,Donkey Kong,127
3,King Dedede,127
4,Ganondorf,118
...,...,...
77,Fox,77
78,Mr. Game & Watch,75
79,Squirtle,75
80,Jigglypuff,68


In [66]:
shield = pd.read_csv('/content/drive/My Drive/SSBU-Tiers/shield.csv', delimiter=';')

shield

,Character,#1,Fastest Move(s),#2,2nd Fastest Move(s),#3,3rd Fastest Move(s),Grab,"Grab, Post-Shieldstun",Item Throw(Forward),Item Throw(Back),Jump+Z-Drop (Front),Jump+Z-Drop(Behind),Unnamed: 13
0,Banjo & Kazooie,9,Usmash,10,Uair,11,Bair,7,11,**,**,**,**,NaN
1,Bayonetta,6,UpB,10,"Fair, SideB (Air)",12,"Nair, Uair",7,11,10,10,4,5,NaN
2,Bowser,6,UpB,9,SideB (Air),11,Nair,8,12,10,13,4,5,NaN
3,Bowser Jr.,7,Usmash,9,Uair,10,Nair,11,15,8,11,4,5,NaN
4,Byleth,9,NAir,11,Up B,13,"UAir, USmash",6,10,**,**,**,**,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,Wolf,10,"Nair, Fair, Uair, DownB (Air)",13,Usmash,15,Jab,6,9,8,11,4,5,NaN
79,Yoshi,6,Nair,8,Uair,11,Usmash,14,18,8,7,4,5,NaN
80,Young Link,7,Nair,8,Uair,9,UpB,12,16,8,10,4,5,NaN
81,Zelda,6,UpB,9,"Nair, Fair, Bair, Usmash",14,NeutralB (Air),10,14,10,10,4,5,NaN
